In [0]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [0]:
from tensorflow.keras.losses import binary_crossentropy, mean_squared_error
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling3D, Activation, UpSampling2D, BatchNormalization, add, concatenate, Lambda, Flatten, Conv2DTranspose,DepthwiseConv2D,ConvLSTM2D,TimeDistributed
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.datasets import fashion_mnist,mnist
import tensorflow.keras as keras 
import tensorflow as tf
import numpy as np 
import cv2 
import os 

In [0]:
def sliding_window(x,y,n_step):
  xs=list()
  ys=list()
  for i in range(len(x)):
    end_idx=i+n_step
    if end_idx>len(x)-1:
      break
    xi=x[i:end_idx,:,:]
    yi=y[i:end_idx]
    xs.append(xi)
    ys.append(yi)
  return np.array(xs),np.array(ys)

In [0]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [0]:
y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)

In [0]:
X_train,Y_train=sliding_window(x_train,y_train,n_step=20)

In [0]:
X_test,Y_test=sliding_window(x_test,y_test,n_step=20)

In [41]:
X_train.shape

(59980, 20, 28, 28)

In [0]:
X_train=X_train.reshape([-1,X_train.shape[1],1,X_train.shape[2],X_train.shape[3]])
X_test=X_test.reshape([-1,X_test.shape[1],1,X_test.shape[2],X_test.shape[3]])

In [0]:
X_val=X_test[:300]
y_val=Y_test[:300]

In [44]:
X_train.shape

(59980, 20, 1, 28, 28)

In [45]:
Y_train.shape

(59980, 20, 10)

In [0]:
def create_model():
  model=keras.models.Sequential([keras.layers.Input(shape=(None,1,28,28)),
                                 ConvLSTM2D(10,kernel_size=(3,3),padding='same',return_sequences=True),
                               BatchNormalization(),
                               ConvLSTM2D(5,kernel_size=(3,3),padding='same',return_sequences=True,stateful=False,data_format='channels_first'),
                               BatchNormalization(),
                               ConvLSTM2D(2,kernel_size=(3,3),padding='same',return_sequences=True,stateful=False,data_format='channels_first'),
                               BatchNormalization(),
                               TimeDistributed(keras.layers.Flatten()),
                                TimeDistributed(keras.layers.Dense(10,))])
  return model

In [52]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model=create_model()
  model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
  model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_9 (ConvLSTM2D)  (None, None, 1, 28, 10)   13720     
_________________________________________________________________
batch_normalization_9 (Batch (None, None, 1, 28, 10)   40        
_________________________________________________________________
conv_lst_m2d_10 (ConvLSTM2D) (None, None, 5, 28, 10)   1100      
_________________________________________________________________
batch_normalization_10 (Batc (None, None, 5, 28, 10)   40        
_________________________________________________________________
conv_lst_m2d_11 (ConvLSTM2D) (None, None, 2, 28, 10)   512       
_________________________________________________________________
batch_normalization_11 (Batc (None, None, 2, 28, 10)   40        
__________________

In [53]:
model.fit(X_train[:30000],Y_train[:30000],batch_size=32,validation_data=[X_val,y_val],epochs=100,verbose=1)

Train on 30000 samples, validate on 300 samples
Epoch 1/100
30000/30000 [==============================] - 191s 6ms/sample - loss: 7.2405 - accuracy: 0.1027 - val_loss: 7.1672 - val_accuracy: 0.1108
Epoch 2/100
28704/30000 [===========================>..] - ETA: 7s - loss: 7.2350 - accuracy: 0.1027

KeyboardInterrupt: ignored